In [1]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

# Задание 1

Необходимо вывести названия треков, которые были приобретены компаниями, которые обслуживаются сотрудниками, нанятыми после 1 сентября 2002 года. 
    

In [2]:
%%sql
SELECT DISTINCT tracks.name as Song, customers.Company as Company, employees.HireDate as HireDate_employee
FROM invoice_items
JOIN invoices ON invoice_items.InvoiceId = invoices.InvoiceId
JOIN customers ON invoices.customerId = customers.customerId
JOIN employees ON customers.SupportRepId = employees.EmployeeId
JOIN tracks ON invoice_items.TrackId = tracks.TrackId
WHERE employees.HireDate > '2002-09-01 0:0:0' and customers.Company IS NOT NULL
limit 10

 * sqlite:///chinook.db
Done.


Song,Company,HireDate_employee
Wet My Bed,JetBrains s.r.o.,2003-05-03 00:00:00
Crackerman,JetBrains s.r.o.,2003-05-03 00:00:00
#9 Dream,JetBrains s.r.o.,2003-05-03 00:00:00
Give Peace a Chance,JetBrains s.r.o.,2003-05-03 00:00:00
Whatever Gets You Thru the Night,JetBrains s.r.o.,2003-05-03 00:00:00
Gimme Some Truth,JetBrains s.r.o.,2003-05-03 00:00:00
De La Luz,JetBrains s.r.o.,2003-05-03 00:00:00
Surrender,JetBrains s.r.o.,2003-05-03 00:00:00
When Evening Falls,JetBrains s.r.o.,2003-05-03 00:00:00
Poprocks And Coke,JetBrains s.r.o.,2003-05-03 00:00:00


## Задание 2
Вывести общую стоимость и продолжительность каждого плейлиста.

In [3]:
%%sql
SELECT playlists.Name as Playlist_name, SUM(tracks.UnitPrice) as Playlist_price, SUM(tracks.Milliseconds) as Duration
FROM tracks
JOIN playlist_track ON playlist_track.TrackId = tracks.TrackId
JOIN playlists ON playlists.PlaylistId = playlist_track.PlaylistId
GROUP BY playlists.PlaylistId, playlists.Name

 * sqlite:///chinook.db
Done.


Playlist_name,Playlist_price,Duration
Music,3257.0999999997507,877683083
TV Shows,423.8700000000012,501094957
90’s Music,1462.2300000000118,398705153
Music,3257.0999999997507,877683083
Music Videos,0.99,294294
TV Shows,423.8700000000012,501094957
Brazilian Music,38.60999999999999,9486559
Classical,74.24999999999999,21770592
Classical 101 - Deep Cuts,24.74999999999999,6755730
Classical 101 - Next Steps,24.74999999999999,7575051


## Задание 3
Для каждого артиста определить все жанры песен, которые у него есть и количество песен по каждому жанру.


In [4]:
%%sql
SELECT artists.name, genres.name, count(tracks.trackid) as Num
FROM tracks
JOIN albums ON albums.albumid = tracks.albumid
JOIN artists ON artists.artistid = albums.artistid
JOIN genres ON genres.genreid = tracks.genreid
GROUP BY artists.artistid, genres.genreid, genres.name, artists.name
limit 10

 * sqlite:///chinook.db
Done.


Name,Name_1,Num
AC/DC,Rock,18
Accept,Rock,4
Aerosmith,Rock,15
Alanis Morissette,Rock,13
Alice In Chains,Rock,12
Antônio Carlos Jobim,Jazz,14
Antônio Carlos Jobim,Latin,17
Apocalyptica,Metal,8
Audioslave,Rock,14
Audioslave,Alternative & Punk,12


## Задание 4
Для каждого артиста найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение общей суммы, заработанной артистом по всем другим жанрам к общей сумме заработанных денег.

In [5]:
%%sql
WITH tab1 as (
SELECT artists.artistid as ID, artists.name as Artists, genres.name as Genre,
    SUM(invoice_items.unitprice * invoice_items.quantity) as SUMMA
-- SUMMA = количество * цена
FROM invoice_items
JOIN tracks ON tracks.trackid = invoice_items.trackid
JOIN albums ON tracks.albumid = albums.albumid
JOIN artists ON albums.artistid = artists.artistid
JOIN genres ON tracks.genreid = genres.genreid
GROUP BY artists.artistid, artists.name, genres.name
),

tab2 as (
SELECT ID, MAX(SUMMA) as M_SUMMA, (1 - MAX(SUMMA) / SUM(SUMMA)) as Relation
FROM tab1
GROUP BY ID
)


SELECT tab1.Artists, tab2.M_SUMMA, tab2.Relation, tab1.Genre
FROM tab1
JOIN tab2 ON tab2.ID = tab1.ID
WHERE tab1.SUMMA = tab2.M_SUMMA AND tab2.ID = tab1.ID
limit 10

 * sqlite:///chinook.db
Done.


Artists,M_SUMMA,Relation,Genre
AC/DC,15.840000000000002,0.0,Rock
Accept,4.95,0.0,Rock
Aerosmith,9.9,0.0,Rock
Alanis Morissette,7.920000000000001,0.0,Rock
Alice In Chains,6.930000000000001,0.0,Rock
Antônio Carlos Jobim,14.850000000000001,0.3181818181818181,Latin
Apocalyptica,3.96,0.0,Metal
Audioslave,5.94,0.625,Rock
BackBeat,5.94,0.0,Rock And Roll
Billy Cobham,3.96,0.0,Jazz


## Задание 5
Для каждой компании вывести общее количестов песен, купленных по жанру рок

In [6]:
%%sql
SELECT customers.company, SUM(invoice_items.quantity) as NUM
FROM customers
JOIN invoices ON customers.customerid = invoices.customerid
JOIN invoice_items ON invoices.invoiceid = invoice_items.invoiceid
JOIN tracks ON invoice_items.trackid = tracks.trackid
JOIN genres ON tracks.genreid = genres.genreid
WHERE customers.company NOT LIKE 'None'
GROUP BY customers.company, genres.name
HAVING genres.name LIKE 'Rock'

 * sqlite:///chinook.db
Done.


Company,NUM
Apple Inc.,15
Banco do Brasil S.A.,11
Embraer - Empresa Brasileira de Aeronáutica S.A.,14
Google Inc.,11
JetBrains s.r.o.,15
Microsoft Corporation,8
Riotur,16
Rogers Canada,12
Telus,13
Woodstock Discos,29


## Задание 6
Вывести альбом, который является минимальным по объему(в байтах) среди альбомов со стоимостью, большей средней цены за альбом по всем альбомам с таким же жанром (если у альбома песни с нескольким жанрами, то жанром является тот, у которого наибольшее количество песен в альбоме)

In [7]:
%%sql
WITH tab1 as(
SELECT albums.Title, albums.albumid, genres.genreid, genres.Name, COUNT(genres.genreid) as Num
FROM tracks
JOIN albums USING(albumid)
JOIN genres USING(genreid)
GROUP BY albums.albumid, genres.genreid),

tab2 as(
SELECT tab1.albumid, tab1.title, tab1.genreid as GenreID, MAX(Num)
FROM tab1
GROUP BY Title
),

tab3 as(
SELECT tab2.GenreID, albums.Title, SUM(tracks.unitprice) as Price
FROM tab2
JOIN albums USING(albumid)
JOIN tracks USING(albumid)
GROUP BY albumid
),

tab4 as(
SELECT AVG(tab3.Price) as AverageCost, tab3.GenreID
FROM tab3
GROUP BY GenreID
),

tab5 as(
SELECT SUM(tracks.unitprice) as AllPrice, SUM(tracks.bytes) as Size, tracks.albumid
FROM tracks
JOIN Albums USING(albumid)
GROUP BY albumid
),

tab6 as(
SELECT tab2.title, tab2.GenreID, MIN(tab5.size) as MinSize
FROM tab2
JOIN  tab4 USING(GenreID)
JOIN  tab5 USING(albumid)
WHERE tab5.AllPrice > tab4.AverageCost
GROUP BY GenreID, Title
)


SELECT Title, GenreID, MIN(MinSize)
FROM tab6

 * sqlite:///chinook.db
Done.


title,GenreID,MIN(MinSize)
English Renaissance,24,3464564
